In [1]:
import pandas as pd

# 파일 경로 지정
hr_path = "../data_csv/heartrate_seconds_merged.csv"
activity_path = "../data_csv/dailyActivity_merged.csv"
sleep_path = "../data_csv/sleepDay_merged.csv"

# CSV 로드
hr = pd.read_csv(hr_path)
activity = pd.read_csv(activity_path)
sleep = pd.read_csv(sleep_path)

# 컬럼 확인
print("../data_csv/heartrate_seconds_merged.csv columns:")
print(hr.columns.tolist(), "\n")

print("../data_csv/dailyActivity_merged.csv columns:")
print(activity.columns.tolist(), "\n")

print("../data_csv/sleepDay_merged.csv columns:")
print(sleep.columns.tolist(), "\n")

../data_csv/heartrate_seconds_merged.csv columns:
['Id', 'Time', 'Value'] 

../data_csv/dailyActivity_merged.csv columns:
['Id', 'ActivityDate', 'TotalSteps', 'TotalDistance', 'TrackerDistance', 'LoggedActivitiesDistance', 'VeryActiveDistance', 'ModeratelyActiveDistance', 'LightActiveDistance', 'SedentaryActiveDistance', 'VeryActiveMinutes', 'FairlyActiveMinutes', 'LightlyActiveMinutes', 'SedentaryMinutes', 'Calories'] 

../data_csv/sleepDay_merged.csv columns:
['Id', 'SleepDay', 'TotalSleepRecords', 'TotalMinutesAsleep', 'TotalTimeInBed'] 



In [3]:
import pandas as pd

base = "../data_csv/merge_전_csv파일/"

hr = pd.read_csv(base + "heartrate_seconds_merged.csv")
act = pd.read_csv(base + "dailyActivity_merged.csv")
slp = pd.read_csv(base + "sleepDay_merged.csv")

act["ActivityDate"] = pd.to_datetime(act["ActivityDate"])
act["date"] = act["ActivityDate"].dt.date

activity_daily = act[[
    "Id", "date", "TotalSteps", "Calories"
]].rename(columns={
    "TotalSteps": "steps",
    "Calories": "calories"
})

print("Activity backbone rows:", len(activity_daily))

hr["Time"] = pd.to_datetime(hr["Time"])
hr["date"] = hr["Time"].dt.date

hr_daily = (
    hr.groupby(["Id", "date"])["Value"]
      .agg(mean_hr="mean", hr_std="std")
      .reset_index()
)

print("HR daily rows:", len(hr_daily))

slp["SleepDay"] = pd.to_datetime(slp["SleepDay"])
slp["date"] = slp["SleepDay"].dt.date

sleep_daily = slp[[
    "Id", "date", "TotalMinutesAsleep"
]].rename(columns={
    "TotalMinutesAsleep": "sleep_minutes"
})

print("Sleep rows:", len(sleep_daily))

merged = activity_daily.merge(
    hr_daily,
    on=["Id", "date"],
    how="left"
).merge(
    sleep_daily,
    on=["Id", "date"],
    how="left"
)

merged = merged.rename(columns={"Id": "user_id"})

print("\nMerged rows:", len(merged))
print("Unique users:", merged["user_id"].nunique())
print("Unique dates:", merged["date"].nunique())

print("\nMissing summary:")
print(merged[["mean_hr", "hr_std", "sleep_minutes"]].isna().sum())



Activity backbone rows: 940
HR daily rows: 143
Sleep rows: 413

Merged rows: 943
Unique users: 33
Unique dates: 31

Missing summary:
mean_hr          935
hr_std           935
sleep_minutes    530
dtype: int64


/var/folders/sc/_dk299z14xs6j6n9ggl_ny5r0000gn/T/ipykernel_71711/1759477183.py:32: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  slp["SleepDay"] = pd.to_datetime(slp["SleepDay"])


In [4]:
merged.to_csv("../data_csv/health_timeseries_core_backbone.csv", index=False)
